In [5]:
# Model Param
TOTAL_TIMESTEP_NUMB = 800_000
LEARNING_RATE = 0.0001
GAE = 1.0
ENT_COEF = 0.01
N_STEPS = 512
GAMMA = 0.9
BATCH_SIZE = 64
N_EPOCHS = 10

# Test Param
EVAL_FREQ = 10000
TEST_EPISODE_NUMBERS = 20

In [6]:
from pathlib import Path

save_dir = Path('./resize')

In [7]:
from model_cnn import MarioNet

policy_kwargs_84 = dict(
    features_extractor_class=MarioNet,
    features_extractor_kwargs=dict(features_dim=512),
)
policy_kwargs_42 = dict(
    features_extractor_class=MarioNet,
    features_extractor_kwargs=dict(features_dim=256),
)
policy_kwargs_21 = dict(
    features_extractor_class=MarioNet,
    features_extractor_kwargs=dict(features_dim=64),
)

In [8]:
from utils import make_parallel_env, STAGE_RECTANGLE

env_84 = make_parallel_env(STAGE_RECTANGLE, 8)
env_42 = make_parallel_env(STAGE_RECTANGLE, 8, resize=42)
env_21 = make_parallel_env(STAGE_RECTANGLE, 8, resize=21)

# dict with the different environments and names
models = {
    'resize_84': dict(env=env_84, policy_kwargs=policy_kwargs_84),
    'resize_42': dict(env=env_42, policy_kwargs=policy_kwargs_42),
    'resize_21': dict(env=env_21, policy_kwargs=policy_kwargs_21),
}

In [ ]:
from utils import TrainAndLoggingCallback
from stable_baselines3 import PPO
for key in models.keys():
    print(f"Training {key}")
    model = PPO('CnnPolicy', models[key]['env'], verbose=0, policy_kwargs=models[key]['policy_kwargs'], tensorboard_log=save_dir,
                learning_rate=LEARNING_RATE, n_steps=N_STEPS, batch_size=BATCH_SIZE, n_epochs=N_EPOCHS, gamma=GAMMA, gae_lambda=GAE, ent_coef=ENT_COEF)
    callback = TrainAndLoggingCallback(test_env=eval, check_freq=EVAL_FREQ, episode_num=TEST_EPISODE_NUMBERS)
    
    model.learn(total_timesteps=TOTAL_TIMESTEP_NUMB, tb_log_name=key, callback=callback)
    
    

Training resize_84
